In [16]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
from konlpy.tag import Okt
from PIL import Image
import pandas as pd
import numpy as np

from tqdm import tqdm
tqdm.pandas()

In [17]:
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import word_tokenize

In [18]:
from konlpy.tag import Okt

In [19]:
#크롤링 데이터 불러오기
#본인이 크롤링한 파일 업로드
data=pd.read_excel('신트리_fps신작리뷰_댓글.xlsx')

#필요없는 컬럼 삭제
del data['Unnamed: 0']
# data.info()
data

,아이디,댓글 내용
0,밉바부,요즘 넥슨은 다양한 시도를 하는거같아서 긍정적으로 보게됩니다. 제발 이대로 좋은 성...
1,그러쿤,베일드 친구들과 너무 즐겁게 플레이했던 게임이었어요. 총 쏘는 맛도 좋고 딱 머리 ...
2,GunHyo Lee,퍼스트 디센던트는 미완성 게임으로 테스트를 지원한거라 생각해서 모션의 어색함이나 박...
3,여심용광로,베엑은 테스트때 참여했던 유저로 게임 정말 잘만들었습니다. 다만 컨텐츠 부족 맵의 ...
4,민트허브,가챠 확률이 영업비밀이 아니라 성공한 신작게임들의 개발 노하우들이 영업비밀이었으면 ...
...,...,...
535,zip,응 피파 미만잡
536,하루,음
537,사과양갤럭씨,노잼처럼 생겼네
538,개발왕,별론데


In [20]:
data.columns=['id', 'comment']
data

,id,comment
0,밉바부,요즘 넥슨은 다양한 시도를 하는거같아서 긍정적으로 보게됩니다. 제발 이대로 좋은 성...
1,그러쿤,베일드 친구들과 너무 즐겁게 플레이했던 게임이었어요. 총 쏘는 맛도 좋고 딱 머리 ...
2,GunHyo Lee,퍼스트 디센던트는 미완성 게임으로 테스트를 지원한거라 생각해서 모션의 어색함이나 박...
3,여심용광로,베엑은 테스트때 참여했던 유저로 게임 정말 잘만들었습니다. 다만 컨텐츠 부족 맵의 ...
4,민트허브,가챠 확률이 영업비밀이 아니라 성공한 신작게임들의 개발 노하우들이 영업비밀이었으면 ...
...,...,...
535,zip,응 피파 미만잡
536,하루,음
537,사과양갤럭씨,노잼처럼 생겼네
538,개발왕,별론데


In [21]:
data.comment = data.comment.str.replace("[^가-힣]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
data.comment = data.comment.str.replace("\s+", " ") # white space 삭제

C:\Users\user\AppData\Local\Temp\ipykernel_11324\2397918913.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data.comment = data.comment.str.replace("[^가-힣]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_11324\2397918913.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data.comment = data.comment.str.replace("\s+", " ") # white space 삭제


In [22]:
data

,id,comment
0,밉바부,요즘 넥슨은 다양한 시도를 하는거같아서 긍정적으로 보게됩니다 제발 이대로 좋은 성적...
1,그러쿤,베일드 친구들과 너무 즐겁게 플레이했던 게임이었어요 총 쏘는 맛도 좋고 딱 머리 아...
2,GunHyo Lee,퍼스트 디센던트는 미완성 게임으로 테스트를 지원한거라 생각해서 모션의 어색함이나 박...
3,여심용광로,베엑은 테스트때 참여했던 유저로 게임 정말 잘만들었습니다 다만 컨텐츠 부족 맵의 종...
4,민트허브,가챠 확률이 영업비밀이 아니라 성공한 신작게임들의 개발 노하우들이 영업비밀이었으면 ...
...,...,...
535,zip,응 피파 미만잡
536,하루,음
537,사과양갤럭씨,노잼처럼 생겼네
538,개발왕,별론데


In [23]:
#결측치 개수 확인
data.isnull().sum()

id         0
comment    1
dtype: int64

In [10]:
import konlpy
kkma = konlpy.tag.Kkma() #형태소 분석기 꼬꼬마(Kkma)

nouns = data['doc'].apply(kkma.nouns)
nouns

0      [요즘, 넥슨, 다양, 시도, 거, 긍정적, 성적, 내서, 한국, 한국게임, 게임,...
1      [베, 베일드, 일드, 친구, 플레이, 게임, 총, 맛, 머리, 정도, 전략, 협동...
2      [퍼스트, 디, 던, 미완성, 게임, 테스트, 지원, 거, 생각, 모션, 어색, 박...
3      [베엑, 테스트, 테스트때, 때, 참여, 유저, 게임, 다, 컨텐츠, 부족, 맵의,...
4       [챠, 확률, 영업, 영업비밀, 비밀, 성공, 신작, 신작게임, 게임, 개발, 노하우]
                             ...                        
533                    [넥, 거름, 돈, 인간, 술, 도박, 경마, 대마, 넥슨]
534                                    [응, 피, 피파, 파, 미만]
535                                                  [음]
536                                           [노, 노잼, 잼]
537                                                  [별]
Name: doc, Length: 538, dtype: object